# BiSeNet v2: Bilateral Network with Guided Aggregation for Real-time Semantic Segmentation  
- 논문 리뷰  

## Abstract  
- low-level detail과 high-level semantic 모두 semantic segmentation task에서 필수적인 요소임  
- 현재 모델들은 inference speed를 위해 low-level detail을 희생함  
- 우리는 spatial detail과 categorical semantic을 따로 분리해 취급한다  

**Architecture**  
- 1. Detail Branch 
    - low level detail을 capture  
    - high resolution feature representation을 생성하기 위해 wide channels과 shallow layers로 구성  
    
- 2. Semantic Branch  
    - high level semantic context를 얻음   
    - narrow channels와 deep layers로 구성  
    
- 3. Guided Aggregation Layer  
    - mutual connection을 강화하고 feature representation을 결합  
    
- BiSeNet v2는 Cityscapes data set에서 156 FPS와 72.6% mIoU의 SOTA를 달성함  

<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure1.JPG?raw=true">

## Introduction   
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure2.JPG?raw=true">  

- semantic segmentation에서 높은 정확도를 달성하는 것은 backbone network와 관련이 있음  
- 크게 2개의 babkbone network 형태가 있다  
    - 1. Dilation backbone  
        - 이는 down sampling 연산을 제거하고 해당 filter kernel을 up sampling을 수행  
        - high resolution feature representation을 얻음  
        
    - 2. Encoder-Decoder backbone  
        - 일반적인 backbone network   
        - top-down과 skip connection을 이용해 decoder part에서 high resolution feature representation을 회복  
        
- 그러나 위의 두 backbone architecture는 inference speed가 느리며 computing cost가 많이 든다  
- real-time semantic segmentation은 빠른 inference 속도를 필요로 함  
- 최근 모델은 이를 위해 2가지 접근법을 사용함  
    - 1. Input Restricting  
        - Input 이미지가 작을수록 computing cost가 적어짐  
    - 2. Channel Pruning  
        - inference 속도를 가속화하며 초기 stage에 channel pruning을 하면 더 빨라짐   
        
- 위의 두 방법은 inference 속도를 어느 정도 개선하지만 low-level detail이 부족함  
- 따라서 높은 효율성과 정확도를 위해 specific architecture를 활용하는 것이 중요함  

- 일반적인 semantic segmentation은 deep하고 wide한 network로 구성됨  
- 이를 이용해 low-level, high-level information 모두를 encoding을 수행함  
- 그러나 본 논문은 spatial detail과 semantic categorical을 별도로 처리해 성능을 높임  
- 즉, two-pathway로 구성된 BiSeNet v2를 제안한다  
- Detail Branch는 wide channels과 shallow layers로 spatial detail을 capture하기 위해 설계됨  
- Semantic Branch는 narrow한 channels과 deep한 layers로 categorical semantic을 추출하기 위해 설계됨  
- Semantic Branch에서는 semantic context를 capture하기 위해 큰 receptive field를 필요로 함  
- detail information은 Detail Branch에서 얻음  
- 따라서 Semantic Branch는 더 적은 channel과 빠른 down-sampling을 수행해 매우 가벼움  
- 이 후, 두 개의 feature representation을 합침  
- 이때 효율적으로 합치기 위해 **Guided Aggregation Layer**를 제안함  
- 또한 inference 성능 향상을 위해 **booster**를 제안함

## Core Concepts of BiSeNet v2  
- 본 논문의 architecture는 크게 Detail Branch, Semantic Branch, Aggregation Layer로 구성돼 있음    

<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure3.JPG?raw=true">

## Detail Branch  
- Detail Branch는 큰 채널 capacity과 풍부한 spatial detailed information을 필요로 함  
- 중요한 concept는 spatial detail을 위해 wide한 channels과 shallow한 layers를 사용하는 것  
- Detail Branch의 feature representation은 큰 spatial size와 wide한 channels를 갖고 있음  
- 따라서 information이 충분하므로 residual connection을 적용하지 않음

## Semantic Branch  
- Semantic Branch는 low channels capacity를 갖고 있음  
- 본 논문에서 어떤 실험을 했는데 각 Branch의 채널 수가 일정 비율의 값을 가지게 해봤다  
- 즉, Semantic Branch가 Detail Branch의 채널의 $\lambda$배가 되도록 해서 lightweight가 되게 함  
- 이때, $\lambda<1$이다  
- Semantic Branch는 feature representation의 level을 증가하고 receptive field를 확대하기 위해 fast down sampling을 적용   
- high level semantic은 large receptive field를 필요로 하므로 GAP를 적용함

## Aggregation Layer  
- 위의 두 Branch의 feature representation은 서로 상호 보완적임  
- Aggregation Layer는 두 feature를 합치기 위해 설계되었음  
- fast down sampling 때문에 Semantic Branch의 output dimension이 Detail Branch의 output dimension보다 작음  
- 그래서 Semantic Branch의 output feature map을 up sampling을 수행해야 함

## Architecture of BiSeNet v2

### Detail Branch  
- Detail Branch는 3 stage로 구성됨  
- 각 stage는 Conv + BN + ReLU로 구성됨    
- first stage의 경우, stride 2 Conv layer를 사용  
- output size는 input size의 1/8  
- VGG의 구조를 따름

### Semantic Branch  
- large receptive field와 효율적인 연산을 위해서 lightweight model을 사용  
- 예를 들면 Xception, MobileNet, ShuffleNet  

<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure4.JPG?raw=true">

**Stem Block**  
- Semantic Branch의 first stage에 해당  
- 2개의 다른 방식으로 down sampling을 수행  
- 이후 두 Branch의 output을 합친다  
- Stem Block는 연산 비용이 효율적임  

**Context Embedding Block**  
- Semantic Branch는 high level semantic을 capture하기 위해서 large receptive filed를 필요로 함  
- global contexual information을 효율적으로 embedding하기 위해 GAP와 resudial connection을 사용함    

<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure5.JPG?raw=true">

**Gather-and-Expansion Layer**  
- depth-wise convolution을 이용함  
- 마지막에 1x1 conv를 이용해 depth-wise conv의 output을 projection시킴  
- 기존 MobileNet v2 구조와 달리 3x3 conv layer를 하나 더 사용해 더 좋은 feature quality를 얻음  

### Bilateral Guided Aggregation  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure6.JPG?raw=true">

- 서로 다른 feature를 합칠 때 여러 방법으로 합치게 됨 (sum, concat 등)  
- 그러나 Detail Branch와 Semantic Branch의 output은 서로 다른 feature level을 가짐  
- 따라서 단순히 합치기만 한다면 좋지 않은 성능이 나올 것임   
- 그래서 두 Branch의 output을 결합하는 Guided Aggregation Layer를 제안한다  
- 이는 Semantic Branch의 contextual한 정보를 이용해 Detail Branch의 feature reponse를 guide한다

### Booster Training Strategy  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure7.JPG?raw=true">

- Segmentation accuracy를 개선시키기 위해 booster training을 제안한다  
- 이는 training 단계에서 feature representation을 향상시키며  
- inference 단계에서 효율을 높일 수 있다  
- 따라서 계산 복잡도가 거의 증가하지 않음  
- 이는 Semantic Branch의 사이사이에 삽입해서 사용함

## Experimental Results  
- Dataset  
    - Cityscapes  
        - train: 2975, valid: 500, test: 1525  
        - class: 30
        - resolution: 2048 x 1024
    - CamVid  
        - train: 367, valid: 101, test: 233  
        - class: 11  
        - resolution: 960 x 720  
    - COCO  
        - train: 9K, test: 1K  
        - class: 91  
        - resolution : 640 x 640
        
- Training  
    - init weight : kaiming normal  
    - optimizer : SGD momentum 0.9  
    - batch size : 16  
    - weight decay : 0.0005 (in Cityscapes and CamVid), 0.0001 (in COCO)  
    - learning rate : 5e-2  
    - lr scheduler : poly learning rate strategy  $\left(1-\frac{iter}{iters_{max}}\right)^{power}$ with power 0.9  
    - iteration : Cityscapes (150K), CamVid (10K), COCO (20K)  
    - data augmentation : random horizontally flip, randomly scale (0.75, 1, 1.25, 1.5, 1.75, 2.0), randomly crop (fixed 2048x1024)  
    
- Inference  
    - metric : mean IoU (Cityscapes and CamVid), pixel accuracy (COCO)  

### Ablative Evaluation on Cityscapes  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure8.JPG?raw=true">    
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/table2.JPG?raw=true">   

- Semantic과 Detain Branch 두 개를 같이 쓰는 것이 성능이 더 좋음

### Aggregation methods  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure9.JPG?raw=true">    
   
   
- Detail Branch는 spatial한 정보를 제공하며   
- Semantic Branch는 semantic context를 capture한다  
- 두 Branch를 같이 사용하게 되면 더욱 세밀한 segmentation을 수행할 수 있음  

<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/table3.JPG?raw=true">


### Channel capacity of Semantic Branch  
- Semantic Branch의 각 채널은 Detail Branch 채널의 $1/\lambda$만큼의 수를 가짐  
- $\lambda=4$가 가장 성능이 좋음  

### Expansion ratio of GE layer  
- expasion ratio $\epsilon$은 GE Layer의 output dimension을 결정함  
- $\epsilon=1$인 경우 baseline보다 mIoU가 4% 이상 개선됨

### Block design of of Semantic Branch  
  

- main improvements  
    - 1. MobileNet v2의 inverted bottleneck에서 1개의 point wise conv 대신 3x3 conv 하나를 Gather Layer로 채택  
    - 2. stride = 2인 경우, 5x5 conv를 한 번 수행하는 것보다 3x3 conv을 두 번 수행하는 것이 더 효율적  

### Booster training strategy  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/figure7.JPG?raw=true">
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/table4.JPG?raw=true">

- 위 그림처럼 Semantic Branch를 훈련할 때는 사이사이에 Booster을 넣고 inference할 때는 폐기  
- Booster 훈련은 정확도를 향상시킴  

### Generalization to large models  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/table5.JPG?raw=true">

- 기존 BiSeNet v2에서 모델을 wider or deeper하게 만들어 실험  
- $\alpha=2$, $d=3$만큼 더 wide하고 deep한 BiSeNet v2 Large 모델을 만듦  
- 그 결과 FPS는 낮아졌지만 정확도가 더 향상됨

## Comparing on dataset  
**Cityscapes data set**

<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/table7.JPG?raw=true"> 

**CamVid data set**
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiSeNet%20v2/figure/table8.JPG?raw=true">

## Conclusion  
- BiSeNet v2는 low-level detail과 high-level semantic 둘 다 뽑아낼 수 있는 모델임  
- 이 모델의 architecture는 일반적이며 범용성이 굉장히 넓음  
- 이 모델은 segmentation accuracy와 inference speed의 trade-off를 적절히 극복함